In [1]:
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from dataloader import CustomDataset
from submission import get_model, eval_transform, team_id, team_name, email_address



In [2]:
evalset = CustomDataset(root='D:/NYU\DeepLearning\hw\Final\dataset', split="val", transform=eval_transform)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=256, shuffle=False, num_workers=2)

net = get_model()
checkPointDir = 'D:/NYU/DeepLearning/hw/Final_git/dl_project/test_DL/model.pth'
checkpoint = torch.load(checkPointDir)
net.load_state_dict(checkpoint)
net = net.cuda()

net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in evalloader:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        #print(torch.max(outputs.data, 1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f"Team {team_id}: {team_name} Accuracy: {(100 * correct / total):.2f}%")
print(f"Team {team_id}: {team_name} Email: {email_address}")

Team 10: dl10 Accuracy: 14.58%
Team 10: dl10 Email: wl2337@nyu.edu


In [8]:
# Feel free to modifiy this file. 
# It will only be used to verify the settings are correct 

import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from dataloader import CustomDataset
from submission import *

# parser = argparse.ArgumentParser()
# parser.add_argument('--checkpoint-dir', type=str)
# args = parser.parse_args()

train_transform = transforms.Compose([
    transforms.ToTensor(),
])


from submission import get_model, eval_transform, team_id, team_name, email_address

unLabeledSet = CustomDataset(root='D:/NYU\DeepLearning\hw\Final\dataset', split="unlabeled", transform=train_transform)
unLabeledLoader = torch.utils.data.DataLoader(unLabeledSet, batch_size = 8, shuffle=False, num_workers = 2)




with torch.no_grad():
    net.eval()
    newlyLabeledList = []
    requestList = []
    for iC, data in enumerate(unLabeledLoader):

        # for data in evalloader:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        #Smallest Margin Uncertainty
        tk_v,tk_i = torch.topk(outputs,2)

        
        
        #margin = torch.max(outputs, 1)[0] - torch.min(outputs, 1)[0]
        margin = tk_v[:,0] - tk_v[:,1]

        maxMarginPosition = torch.argmax(margin)
        pickedUnlabelled_img = images[maxMarginPosition]
        pickedUnlabelled_lbl = tk_i[maxMarginPosition][0]

        names_start = iC*8
        
        minScore = torch.argmin(margin)
        if(minScore==maxMarginPosition):
            continue
        else:
            request = names_start + minScore
        #print(pickedUnlabelled_img)
        tempTouple = (names_start+maxMarginPosition, pickedUnlabelled_lbl,)
        #print(tempTouple)
        newlyLabeledList.append(tempTouple)
        #print(pickedUnlabelled_img.tolist())
        requestList.append(request)





In [9]:
len(newlyLabeledList)

64000

In [10]:
len(requestList)


64000

In [11]:
with open('request_10.csv', 'w') as f:
    for idx in requestList:
        #print(idx)
        if(idx.item()/8 >= 12800): break
        print(f"{idx}.png",file = f)
 

In [20]:
with open('newlyLabeled.txt', 'w') as f:
    for tempTouple in newlyLabeledList:
        name,label = tempTouple
        print(f"{name.item()}.png,{label.item()}",file = f)


In [21]:
labelDir = r"D:\NYU\DeepLearning\hw\Final\dataset\train_label_tensor.pt"

In [22]:
givenlabels = torch.load(labelDir)

In [23]:
givenlabels

tensor([289, 289, 289,  ...,  93,  93,  93])

In [24]:
with open('movedFileNames.txt', 'w') as f:
    for tempTouple in newlyLabeledList:
        name,label = tempTouple
        print(f"{name.item()}.png",file = f)

In [25]:
with open('movedFileLabels.txt', 'w') as f:
    for tempTouple in newlyLabeledList:
        name,label = tempTouple
        print(f"{label.item()}",file = f)

In [4]:
labelDir = r"D:\NYU\DeepLearning\hw\Final_git\dl_project\test_DL/label_10.pt"

In [43]:
newlyLabeledList

[(tensor(2, device='cuda:0'), tensor(125, device='cuda:0')),
 (tensor(12, device='cuda:0'), tensor(107, device='cuda:0')),
 (tensor(18, device='cuda:0'), tensor(678, device='cuda:0')),
 (tensor(30, device='cuda:0'), tensor(395, device='cuda:0')),
 (tensor(35, device='cuda:0'), tensor(556, device='cuda:0')),
 (tensor(43, device='cuda:0'), tensor(545, device='cuda:0')),
 (tensor(53, device='cuda:0'), tensor(770, device='cuda:0')),
 (tensor(61, device='cuda:0'), tensor(54, device='cuda:0')),
 (tensor(71, device='cuda:0'), tensor(371, device='cuda:0')),
 (tensor(79, device='cuda:0'), tensor(399, device='cuda:0')),
 (tensor(85, device='cuda:0'), tensor(350, device='cuda:0')),
 (tensor(91, device='cuda:0'), tensor(419, device='cuda:0')),
 (tensor(100, device='cuda:0'), tensor(747, device='cuda:0')),
 (tensor(106, device='cuda:0'), tensor(269, device='cuda:0')),
 (tensor(118, device='cuda:0'), tensor(328, device='cuda:0')),
 (tensor(124, device='cuda:0'), tensor(433, device='cuda:0')),
 (tens

In [53]:
output_pt = []
for tempTouple in newlyLabeledList:
    name,label = tempTouple
    output_pt.append(label.item())

In [1]:
output_pt = torch.tensor(output_pt)
output_pt

NameError: name 'torch' is not defined

In [57]:
torch.save(output_pt,labelDir)

In [5]:
temp = torch.load(labelDir)

In [6]:
temp

tensor([ 70,  93, 612,  ..., 274, 466, 260])

torch.Size([12800])